<a href="https://colab.research.google.com/github/gauravbhalar95/Telegram_bot/blob/main/Recipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

!pip install pyTelegramBotAPI google-api-python-client oauth2client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 923.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 2.8 MB/s eta 0:00:00


In [ ]:

import telebot
import googleapiclient.discovery
from oauth2client.service_account import ServiceAccountCredentials
from telebot import types
import time
import threading

# Replace with your bot's token
Token = "7316963125:AAH_zNBZkaguB4pD8rBmadWg1Iwm9Efm_BI"

bot = telebot.TeleBot(Token)

# Blogger API setup
BLOG_ID = "6506022432368434594"  # Replace with your Blogger blog ID
SCOPES = ['https://www.googleapis.com/auth/blogger']

# Load credentials from the credentials.json file
credentials = ServiceAccountCredentials.from_json_keyfile_name('/content/drive/MyDrive/Json/spiceloversdelight-586791ec0797.json', scopes=SCOPES)

blogger_service = googleapiclient.discovery.build('blogger', 'v3', credentials=credentials)

# Function to fetch all blog posts with pagination
def fetch_all_blog_posts():
    posts = []
    next_page_token = None
    while True:
        request = blogger_service.posts().list(blogId=BLOG_ID, pageToken=next_page_token, fetchBodies=True)
        response = request.execute()
        posts.extend(response.get('items', []))
        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
    return posts

# Function to fetch blog posts based on a query
def fetch_blog_posts(query):
    request = blogger_service.posts().list(blogId=BLOG_ID, q=query, fetchBodies=True)
    response = request.execute()
    return response.get('items', [])

# Start command handler
@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.reply_to(message, "Welcome to the Recipe Bot! 🍽️\nType 'list' to view available recipes or type an ingredient/recipe name to search.")

# List recipes handler
@bot.message_handler(func=lambda message: message.text.lower() == "list")
def list_recipes(message):
    posts = fetch_all_blog_posts()

    if posts:
        markup = types.InlineKeyboardMarkup()
        for post in posts:
            title = post.get('title', 'No title')
            url = post.get('url', '')
            markup.add(types.InlineKeyboardButton(text=title, url=url))

        bot.send_message(message.chat.id, "Here are the available recipes:", reply_markup=markup)
    else:
        bot.reply_to(message, "Sorry, I couldn't find any recipes.")

# Handler to search and display posts based on user query
@bot.message_handler(func=lambda message: True)
def handle_message(message):
    query = message.text.lower()
    posts = fetch_blog_posts(query)

    if posts:
        post = posts[0]  # Get the first matching post
        title = post.get('title', 'No title')
        snippet = post.get('content', 'No content')[:200]  # Get a snippet of the content
        url = post.get('url', '')

        response = f"**{title}**\n\n{snippet}...\n\nRead more: {url}"
    else:
        response = "Sorry, I couldn't find any recipes matching that query."

    bot.reply_to(message, response)

# Function to check for new posts
def check_for_new_posts(last_known_post_id):
    new_posts = fetch_all_blog_posts()
    latest_post_id = new_posts[0].get('id') if new_posts else None
    if latest_post_id != last_known_post_id:
        notify_new_post(new_posts[0])
        return latest_post_id
    return last_known_post_id

# Notify user about the new post
def notify_new_post(post):
    title = post.get('title', 'No title')
    url = post.get('url', '')
    message = f"New Recipe Posted! 🍽️\n\n**{title}**\nRead more: {url}"
    bot.send_message(chat_id='-1001524250802', text=message)  # Replace with your chat or channel ID

# Periodically check for new posts
def periodic_check():
    last_known_post_id = None
    while True:
        last_known_post_id = check_for_new_posts(last_known_post_id)
        time.sleep(600)  # Check every 10 minutes

# Start the periodic check in a separate thread
threading.Thread(target=periodic_check).start()

# Start polling
bot.infinity_polling()

Exception in thread Thread-13 (periodic_check):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-10-97ce82d0dc08>", line 100, in periodic_check
  File "<ipython-input-10-97ce82d0dc08>", line 85, in check_for_new_posts
  File "<ipython-input-10-97ce82d0dc08>", line 94, in notify_new_post
  File "/usr/local/lib/python3.10/dist-packages/telebot/__init__.py", line 1766, in send_message
    apihelper.send_message(
  File "/usr/local/lib/python3.10/dist-packages/telebot/apihelper.py", line 273, in send_message
    return _make_request(token, method_url, params=payload, method='post')
  File "/usr/local/lib/python3.10/dist-packages/telebot/apihelper.py", line 168, in _make_request
    json_result = _check_result(method_name, result)
  File "/usr/local/lib/python3.10/dist-packages/telebot/apih